# Intro to Spark

**Disclaimer:** TP plus compliqué !

**Spark** est un système de calcul hautement parallélisé :
- au niveau du stockage : la data est fragmentée, dupliquée, répartie sur un nombre quelconque de disques/servers
- au niveau du calcul : les calculs s'exécutent en parallèle sur plusieurs machines, chacune sur sa portion de data

Spark en tant que tel est un framework qui a des implémentation dans plusieurs langages :
- Python via PySpark (ce qe nous utiliserons)
- Scala
- R
- Java

## Quelques informations en vrac sur Spark
### Hardware
- Spark peut s'installer sur un grand nombre de machines situées dans un même réseau. Elles pourront ensuite se reconnaître et collaborer. 1 unité de calcul = 1 noeud.
- Spark fonctionne sur le mode master/worker : un noeud est désigné `master` et jouera le rôle de chef d'orchestre pour que les autres noeuds `workers` exécutent les tâches dans le bon ordre
- Les noeuds Spark communiquent énormément entre eux pour s'échanger des informations et surtout des données

### Software
- Un code Spark / PySpark doit utiliser les primitives Spark pour que tout s'exécute selon la logique Spark
- Le code pyspark est transmis au noeud `master` qui le lit et prépare l'orchestration des calculs selon les `workers` qu'il a à disposition. Seuls les `workers` manipuleront la donnée (sauf exception)
- Spark est *lazy* : `master` ne lance réellement aucun calcul tant qu'il n'a pas lu d'opération impliquant l'affichage ou l'écriture des résultats
- Corrolaire du *lazy* : sans précaution, Spark peut répéter plusieurs fois les mêmes calculs ... Exemple avec 2 chaînes de transformation data `A -> B -> C -> D` suivi de `A -> B -> E`. Les étapes intermédiaires `A -> B` sont identiques mais pour calculer `D` et `E`, Spark risque de les exécuter 2 fois. Apprendre à manipuler les méthodes [cache](https://spark.apache.org/docs/3.5.3/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.cache.html?highlight=cache) !

### Framework
- Spark se base sur des `DataFrame` très proches en terme d'utilisation des `pandas.DataFrame` donc pas de panique :)
- Spark gère tout ce que sait gérer SQL mais part de fichiers plats (ici CSV) : join, select, etc ...
- PySpark permet de gérer tout ces concepts avec la facilité d'accès du Python
- PySpark est TRÈS typé et a besoin de connaître les types de chaque colonne manipulées
- Spark est TRÈS flexible en terme de configuration et d'exécution et cela peut sembler déroutant pour des exemples simples

### Organisation du calcul
En informatique, à chaque architecture ses optimisations :
- en local sur 1 CPU, un calcul possède peu d'optimisation : simple et linéaire, possiblement async
- en local sur multi CPU, un calcul doit être prévu pour paralléliser les exécutions sur plusieurs coeurs physiques
- en local sur mono/multi GPU, un calcul doit être hautement parallélisable, découpable en tranche de data qui tiennent en GPU-RAM
- en multi machine multi GPU, idem que plus haut avec bande passante importante entre machine pour échange d'information

... Spark gère le multi machine, multi CPU, multi RAM, multi disque : calculs hautement parallélisés grâce à la magie de Spark, data échangées au mieux entre machine (possiblement avec l'aide humaine).
__Spark est toujours prêt à gérer un contexte d'exécution très complexe__ => il faut s'attendre à beaucoup d'overhead sur des cas simples

**Les opérations s'exécutent sur des workers séparés, en parallèle**, il faut donc parfois faire "un peu attention" à la façon dont on demande à Spark de *partitionner* sa data.

## À retenir

1. Spark et son implémentation PySpark sont très puissant car gèrent un parallélisme quasi infini et réglable à 100%
2. PySpark a un coût d'entrée pour se couler dans le moule Spark mais permet de réaliser des opérations très complexes avec la simplicité du Python
3. Votre notebook n'exécutera n'a pas accès à la data manipulée et n'effectuera aucun calculs ; il les transmettra au Spark Master qui les répartira entre ses workers qui ont accès à la data

## Exemple de code Spark

In [ ]:
import langdetect

In [ ]:
import requests

def get_embedding(text: str):
    url = "http://embedder:8000/embed"
    payload = {"text": text}
    
    response = requests.post(url, json=payload)
    response.raise_for_status()  # Raise an exception for HTTP errors
    return response.json()

In [ ]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np

In [ ]:
MY_ID = "François"

In [ ]:
# Initialize PySpark session
assert MY_ID is not None, "provide your id first!"
spark = SparkSession.builder \
    .appName(MY_ID) \
    .master("spark://spark-master:7077") \
    .config("spark.executor.memory", "1g") \
    .config("spark.dynamicAllocation.enabled", "true") \
    .config("spark.executor.cores", "1") \
    .config("spark.executor.instances", "1") \
    .getOrCreate() 

# /!\ Tout se fera à partir de cet object magique `spark`

In [ ]:
%%time
data = [("Alice", 1), ("Bob", 2), ("Catherine", 3)]
df = spark.createDataFrame(data, ["Name", "Value"])

# Show the dataframe
df.show()

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import StringType, FloatType, ArrayType, StructField, StructType, DoubleType

In [ ]:
df_beers = spark.read.csv("/datasets/csv/beers.csv", header=True)
df_beers.show()

In [ ]:
%%time
# Define a specific funtion to map beer names
@F.udf(returnType=StringType())
def revert_cap_name(name: str):
    return name[::-1].upper()


df_beers.repartition(12).withColumn("reverse_capitalized", revert_cap_name(F.col("name"))).select("name", "reverse_capitalized").show()

## Observations
Que remarque-t-on tout de suite ?

# Uses cases

# UC-1 : description data

- Q1: Combien y a-t-il de bières dans la DB ?
- Q2: Top10 brasseries les plus représentées avec le nombre de bière par brasserie ?
- Q3: Top10 des bières les plus fortes (ABV) en France ?
- Q4: Par pays, nombre de brasseries qui proposent des bières de type `Porter` et ABV moyen de celles-ci ?
- Q5: Mediane du nombre de bière par pays ?

In [ ]:
# Q1
df_beers = spark.read.csv("/datasets/csv/beers.csv", header=True)
df_beers.count()

In [ ]:
#Q2 
df_breweries = spark.read.csv("/datasets/csv/breweries.csv", header=True)

In [ ]:
df_beers.filter(F.col("name").ilike("Ho%")).toPandas()

In [ ]:
%%time
df_beer_brew = (
    df_breweries.join(df_beers, on=df_beers.brewery_id == df_breweries.id)
    .select(df_breweries.name)
    .groupby(F.col("name"))
    .count()
    .sort("count", ascending=False)
    .limit(10)
).show()

In [ ]:
df_beers.filter(F.col("name").ilike("Belz%")).show()

In [ ]:
# Q3 top 10 bières FR les plus fortes
df = (
    df_beers.join(df_breweries, on=df_beers.brewery_id == df_breweries.id)
    .filter(F.col("country") == F.lit("France"))
    .select([df_beers.name, df_beers.abv.cast(FloatType())])
    .sort("abv", ascending=False)
    .limit(10)
).show()

In [ ]:
df_style = spark.read.csv("/datasets/csv/styles.csv", header=True)
df_style.show()

In [ ]:
(
    df_style
    .select(df_style.id.alias("style_id"), "style_name")
    .filter(F.col("style_name").ilike("%Porter%"))
).show()

In [ ]:
@F.udf(returnType=FloatType())
def safe_cast_to_float(str_float: str):
    return float(str_float)

df_beers_brewers = (
    df_beers
    .join(df_breweries.withColumnRenamed("name", "brewer_name"), on=df_beers.brewery_id == df_breweries.id)
).cache()

In [ ]:
%%time
print("Q4")
df_style = spark.read.csv("/datasets/csv/styles.csv", header=True)
target_style_id = df_style.filter(F.lower(F.col("style_name")) == "porter").select(F.col("id").alias("style_id"))
dd = (
    df_beers_brewers
    .join(target_style_id, how="inner", on="style_id")
    .withColumn("abv_float", safe_cast_to_float(F.col("abv")))
    .select(["name", "brewer_name", "abv_float", "country"])
    .groupby("country")
    .agg(F.avg("abv_float").alias("avg_abv"), F.countDistinct("brewer_name").alias("n_brewer_having_porter"))
    .show()
)

In [ ]:
%%time
print("Q5")
dd = (
    df_beers_brewers
    .groupby("country")
    .count()
    .agg(F.median("count"))
)
print("Q5:", dd.first()[0])

# UC-2 : préparer un dataset de ranking 
Tout moteur de recherche/search-engine - **SE** - nécessite de la configuration ... beaucoup de configuration. Une des configuration très orientée "data" est le calcul que l'index doit opérer pour scorer chaque réponse possible face à une requête. L'apprentissage statistique de ce score s'appelle *Learning to Rank*  - **LTR** - et nécessite des connaissances poussées en machine learning. 

Cette tâche LTR se base sur les *feedbacks implicites* des utilisateurs face au moteur de recherche. Commençons par un exemple. Quand vous cherchez un objet sur LeBonCoin, vous laissez plusieurs informations *implicites* sur votre perception des résultats proposés : les item sur lesquels vous avez cliqués bien sûr mais également ceux que vous avez probablement *vu* sans cliquer dessus ... Ces "vues sans clics" sont une précieuse information implicite sur les jugement que vous avez porté aux résultats proposés. Pour ce TP nous nous limiterons à ce concept de "vu x click" mais il est possible d'aller plus loin (dwell-time, hierarchisation des interactions explicites, ...). 

On appelle *Search Engine Results Page* - **SERP** - la liste des résultats classés par un SE. Un document qui figure dans les résulats d'une recherche a donc une position (son rang) au sein de la **SERP**.

Exemple, où :
- `query` est la recherche réalisée par un user et qui a débouché sur une SERP
- `clicked_id` : l'id de la bière cliquée par le user
- `user_id` l'id de l'utilisateur (simplifions en disant que c'est même l'id d'une recherche) : permet de retrouver tous les résultats proposés dans **une** recherche
- `id_in_serp` : l'id d'une bière figurant dans la SERP
- `pos_in_serp` : la position/le rang de la bière `id_in_serp` dans la SERP issue de la recherche 

In [ ]:
df_pref = spark.read.csv("/datasets/beers_feedback.csv", header=True, inferSchema=True)
df_pref.limit(3).show()

Un travail préliminaire au LTR est la constitution d'un dataset qui permet d'aggréger ces feedbacks laissés par tous les utilisateurs ayant réalisé la même query. Chacun a vu et cliqué selon ses propres impressions de pertinence et il convient de "moyenner" tout cela pour obtenir des appréciations globales. L'objectif d'un tel dataset est de pouvoir lister des exemples de triplets `(query, document, note)` qui permet de savoir que face à une *query* `milky stout low bitterness`, un *document* `Super bitter beer brewed with organic roasted barley and chocolate` aura une pertinence de *1/4* (arbitraire). 

Implémenter le modèle d'agrégation de feedback "cascade model" [1] (pour la culture, **inutile d'avoir lu l'article** pour le TD) qui propose une approche pragmatique pour obtenir ces données. La méthode est la suivante :
- pour chaque recherche utilisateur:
    - étudier la position de l'id cliqué dans la SERP - soit `clicked_pos_in_serp` cette information
    - Considérer que tout doc situés "au-dessus dans la SERP" (càd quand `pos_in_serp <= clicked_pos_in_serp`) avait été vu par l'utilisateur
    - Récapituler tous ces documents "vus et cliqués" et "vus mais pas cliqués"
- Pour chaque recherche et bière cliquée (`clicked_id`), calculer la "probabilité de clic sachant qu'elle a été vue", càd le nombre de fois qu'elle a été cliquée divisé par le nombre de fois où elle a été vue


[1] https://dl.acm.org/doi/abs/10.1145/1341531.1341545

In [ ]:
# output schema : https://stackoverflow.com/a/54771215/10716281

from pyspark.sql.types import *

mapping = {
    "float64": DoubleType,
    "object":StringType,
    "int64":IntegerType,
    "int32":IntegerType,
    "bool": BooleanType,
} # Incomplete - extend with your types.

def createUDFSchemaFromPandas(dfp):
  column_types  = [StructField(key, mapping[str(dfp.dtypes[key])]()) for key in dfp.columns]
  schema = StructType(column_types)
  return schema

def compute_cascade_model_per_user_query(df: pd.DataFrame) -> pd.DataFrame:
    pos_of_clicked_id = df[df["id_in_serp"] == df["clicked_id"]].iloc[0]["pos_in_serp"]
    df["seen"] = (df["pos_in_serp"] <= pos_of_clicked_id).astype(int)
    df["clicked"] = np.where(df["pos_in_serp"] == pos_of_clicked_id, 1, 0)
    return df

In [ ]:
df_processed = compute_cascade_model_per_user_query(df_pref.limit(3).toPandas())
schema = createUDFSchemaFromPandas(df_processed)

In [ ]:
%%time
(
    df_pref
    .repartition(12, "query", "user_id")
    .groupby(["query", "user_id"])
    .applyInPandas(compute_cascade_model_per_user_query, schema)
    .filter(F.col("seen") == F.lit(1))
    .groupby(["query", "id_in_serp"])
    .agg(F.sum("seen").alias("n_seen"), F.sum("clicked").alias("n_clicked"))
    .withColumn("clic_proba", F.col("n_clicked") / F.col("n_seen"))
    .select(["query", "id_in_serp", "clic_proba"])
    .sort(["query", "id_in_serp"], ascending=False)
    .show()
)

# UC-3.0 Génération de keywords pour chaque bière

In [ ]:
corpus = df_beers.filter(F.col("descript").isNotNull()).withColumn("descript_lower", F.lower("descript")).select("id", "descript_lower")
corpus.show()

In [ ]:
df_words_count_per_beer_id = (
    corpus
    .withColumn("words", F.explode(F.split(F.col("descript_lower"), " ")))
    .select("id", "words")
    .groupby("id", "words")
    .count().withColumnRenamed("count", "term_freq_in_doc")
).cache()
df_words_count_per_beer_id = (
    df_words_count_per_beer_id
    .groupby("words")
    .sum().withColumnRenamed("sum(term_freq_in_doc)", "term_freq_in_corpus")
    .join(df_words_count_per_beer_id, on="words")
    .filter(F.col("term_freq_in_corpus") >= F.lit(2))
    .withColumn("tfidf", F.col("term_freq_in_doc") / F.col("term_freq_in_corpus"))
)
df_words_count_per_beer_id.show()

In [ ]:
df_words_count_per_beer_id.sort("tfidf", ascending=False).show()

In [ ]:
from pyspark.sql.window import Window

In [ ]:
window_spec = Window.partitionBy("id").orderBy(F.col("tfidf").desc())

In [ ]:
(
    df_words_count_per_beer_id
    .withColumn("ranked", F.row_number().over(window_spec))
    .filter(F.col("ranked") <= F.lit(3))
    .groupby("id")
    .agg(F.collect_list("words"))
    .withColumnRenamed("collect_list(words)", "keywords")
).show()

In [ ]:
df_beers.show()

## UC-3.1 langdetect over descripts

In [ ]:
#@F.udf(returnType=StringType())
def detect_lang(text):
    try:
        return langdetect.detect(text)
    except langdetect.LangDetectException:
        return "ukn"

spark_langdetect = F.udf(detect_lang, returnType=StringType())

In [ ]:
(
    df_beers
    .filter(F.col("descript").isNotNull())
    .select("id", "descript")
    .withColumn("lang", spark_langdetect(F.col("descript")))
).show()

In [ ]:
df = df[df.descript.str.len() >= 3]

In [ ]:
def detect_lang(text):
    try:
        return langdetect.detect(text)
    except langdetect.LangDetectException:
        return "ukn"

In [ ]:
df["lang"] = df.descript.apply(detect_lang)

In [ ]:
df.groupby("lang").count()

# UC-3 : récupérer les docs qui parlent d'un mot

Peut-on utiliser SQL pour réaliser un mini moteur de recherche ? Pour différentes requêtes (`query` en anglais) textuelles très simples à base de mot-clef, retrouver les bières qui semblent répondre à la demande. Exemples :
- trouver les bières ou les brasseries qui parlent de bières "fine"
- idem pour "juicy"
- idem pour "genuine"
- idem pour les bières mâturées dans des "oak cask" (fûts en chêne) -> combien y en a-t-il ? $N_1$
   - idem pour les bières qui évoquent uniquement "cask" -> combien y en a-t-il ? $N_{1,1}$
   - idem pour celles ne parlant que de "oak" -> combien y en a-t-il ? $N_{1,2}$
- idem pour les bières qui évoquent "oak" et "cask" -> combien y en a-t-il ? $N_{2}$

In [ ]:
# your code

# UC-4 : vectorisation des description des bières
Préparer le recours à un service de vectorisation qui permettra de convertir la connaissance sur une bière en un vecteur numérique. Ce vecteur permet de sythétiser mathématiquement l'information disponible sur une bière et sa brasserie et pourra être réutilisé plus tard dans un moteur de recherche.
à faire :
- Préparer une description la plus complète possible pour chaque bière
- envoyer ces descriptions une à une via un appel HTTP sur Jina (voir instruction plus bas)

**Découpez le travail** : chacun travaillera sur un sous-ensemble de bières selon l'`id` de chaque bière `beers.id`. 
Vous êtes 12, je propose donc la répartition suivante :
- ADAM.LUCAS --> s'occuper des `beers.id` égaux à 0 modulo 12
- ALIEINIK.OLHA --> s'occuper des `beers.id` égaux à 1 modulo 12
- ARNOUT.FABRICE --> s'occuper des `beers.id` égaux à 2 modulo 12
- BEDIER.DORIANE --> s'occuper des `beers.id` égaux à 3 modulo 12
- CASTRO.MOUCHERON --> s'occuper des `beers.id` égaux à 4 modulo 12
- COLIN.KEVIN --> s'occuper des `beers.id` égaux à 5 modulo 12
- FRASELLE.NADEGE --> s'occuper des `beers.id` égaux à 6 modulo 12
- KUKSA.OLEKSANDRA --> s'occuper des `beers.id` égaux à 7 modulo 12
- LOPES.VAZ.ALEXIS --> s'occuper des `beers.id` égaux à 8 modulo 12
- REITER.ROMAIN --> s'occuper des `beers.id` égaux à 9 modulo 12
- RICHIER.MARCUS --> s'occuper des `beers.id` égaux à 10 modulo 12
- VINOT.MATHIEU --> s'occuper des `beers.id` égaux à 11 modulo 12

## Service de vectorisation Jina
Nous allons faire appel à un service de vectorisation externe [https://jina.ai](https://jina.ai) qui propose gratuitement 1M token de vectorisation. 
Quand vous voudrez vectoriser un texte, suivez la doc de [https://jina.ai/embeddings/](https://jina.ai/embeddings/). 

Nous utiliserons **TOUS le MÊME modèle d'embedding** : `jina-embeddings-v2-base-en` ! Faites donc attention à appeler le bon

Essayons de construire d'avoir tous le même schéma de texte à vectoriser :
`the beer BEER_NAME from brewery BREWERY_NAME (BREWERY_DESCRIPTION) is defined as BEER_DESCRIPTION. Spec of the beer are: ABV=ABV_VALUE, IBU=IBU_VALUE, SRM=SRM_VALUE`

#### Instructions pour appeler le service Jina
En plus de la doc sur leur site, voici un snippet de code:

In [ ]:
import requests

def get_embedding(text: str):
    url = "http://embedder:8000/embed"
    payload = {"text": text}
    
    response = requests.post(url, json=payload)
    try:
        response.raise_for_status()  # Raise an exception for HTTP errors
        return response.json()["vector"]
    except:
        return []

spark_get_embedding = F.udf(get_embedding, returnType=ArrayType(FloatType()))

In [ ]:
(    df_beers
    .repartition(32)
    .filter(F.col("descript").isNotNull())
    .select("id", "descript")
).count()

In [ ]:
%%time
df = (
    df_beers
    .repartition(32)
    .filter(F.col("descript").isNotNull())
    .select("id", "descript")
    .withColumn("lang", spark_get_embedding(F.col("descript")))
)
df.show()

In [ ]:
df.schema

In [ ]:
# your code

# UC-5 : answer question in corpa

**Question difficile en Spark**

**Grandes lignes :** trouvons les documents qui répondent à une question. Exemple : à partir de la description vectorisée à UC-4 pour chaque bière, comment trouver les bières qui répondent à une description plus complète ? Exemple:
- "very bitter beer with smoky taste"
- "fruity sour - balanced sourness"
- "weird beer"

Voir la doc [Spark ML lib - feature extraction](https://spark.apache.org/docs/latest/api/python/reference/pyspark.mllib.html#feature) pour trouver des idées (TF-IDF, Word2Vec), ou utiliser le résultats de vos vectorisation de UC-4.

In [ ]:
queries = ["very bitter beer with smoky taste", "fruity sour - balanced sourness", "weird beer"]